In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

sklearn version:  0.24.2
TF version:  2.8.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


In [2]:
X_train=np.uint8(np.load('./X_train_aug.npy'))
y_train=np.uint8(np.load('./y_train_aug.npy'))
X_valid=np.uint8(np.load('./X_valid.npy'))
y_valid=np.uint8(np.load('./y_valid.npy'))

In [3]:
X_train.shape,X_valid.shape

((1074263, 32, 32, 1), (164015, 28, 28))

In [4]:
X_valid=tf.reshape(X_valid,[-1,28,28,1])
X_train=tf.reshape(X_train,[-1,32,32,1])
X_valid_resize=np.uint8(tf.image.resize(X_valid, [32, 32]))
X_train_resize=np.uint8(tf.image.resize(X_train, [32, 32]))
X_train_resize.shape,X_valid_resize.shape,X_train.dtype

((1074263, 32, 32, 1), (164015, 32, 32, 1), tf.uint8)

In [5]:
X_train=keras.applications.mobilenet_v2.preprocess_input(np.array(X_train,np.float32))
X_valid_final=keras.applications.mobilenet_v2.preprocess_input(np.array(X_valid_resize,np.float32))

In [6]:
np.min(X_train)

-1.0

In [7]:
#황성현 전민재 He_normal initializer
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            
            ]
        self.strides_layers=[
           ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n
        
    def build(self,batch_input_shape):
        print(self.name,": batch_input_shape =",batch_input_shape)
        initializer = tf.keras.initializers.HeNormal()
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1, kernel_initializer=initializer,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=1,padding="SAME", activation=self.activation,kernel_initializer=initializer),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c,kernel_size=1, strides=1,padding="SAME", activation=self.activation,kernel_initializer=initializer),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1,kernel_initializer=initializer,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=self.s,padding="SAME",activation=self.activation,kernel_initializer=initializer),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c, kernel_size=1,strides=1,padding="SAME",activation=self.activation,kernel_initializer=initializer),
        ]

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [12]:
    #황성현 전민재 IRB6512
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=2,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=4,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=4,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=4,c=512,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
    batch_size=128
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [8]:
#전민재
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)
    def on_epoch_end(self, batch, logs):
        print("lr: {}".format(self.model.optimizer.lr))

In [13]:
#황성현 전민재
from datetime import datetime
K = keras.backend
epochs=40
logs = "logs/" + datetime.now().strftime("smv2l_adam_best_lrsch_onecycle_aug_dropout_128_he_irb6_512_t4_epoch40%Y%m%d-%H%M%S")
print(logs)
onecycle = OneCycleScheduler(len(X_train_resize) // batch_size * epochs, max_rate=0.003)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_best_lrsch_onecycle_aug_dropout_128_he_irb6_512_t4_epoch40.h5",save_best_only=True)

history=smv2_l.fit(X_train,y_train,
                   validation_data=(X_valid_final,y_valid),batch_size = batch_size, 
                   epochs=epochs,callbacks=[tboard_callback, check_best_cb,onecycle])

logs/smv2l_adam_best_lrsch_onecycle_aug_dropout_128_he_irb6_512_t4_epoch4020220608-172445


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [23]:
smv2_l_best=keras.models.load_model("smv2l_adam_best_lrsch_onecycle_aug_dropout_128_he_irb6_512_t4_epoch40.h5",
                                   custom_objects={"Inveted_Residual_Block":Inveted_Residual_Block})

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)


In [24]:
smv2_l_best.evaluate(X_valid_final,y_valid)

5126/5126 [==============================] - 29s 6ms/step - loss: 0.3335 - accuracy: 0.8773


[0.3335278034210205, 0.8773344159126282]

In [25]:
smv2_l_accbest=keras.models.load_model("smv2l_adam_lrsch_onecycle_aug_dropout_128_he_irb6_512_epoch40.h5",
                                   custom_objects={"Inveted_Residual_Block":Inveted_Residual_Block})

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)


In [27]:
smv2_l_accbest.evaluate(X_valid_resize,y_valid)

5126/5126 [==============================] - 34s 7ms/step - loss: 0.3293 - accuracy: 0.8784


[0.3292771875858307, 0.8784440159797668]

In [9]:
    #황성현 전민재 IRB6512, maxlr 0.002
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=2,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=4,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=4,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=4,c=512,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
    batch_size=128
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 32, 32, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)     

In [10]:
#황성현 전민재
from datetime import datetime
K = keras.backend
epochs=60
logs = "logs/" + datetime.now().strftime("SMV2L_lrsch_onecycle_dropout_128_he_irb6_512_epoch40%Y%m%d-%H%M%S")
onecycle = OneCycleScheduler(len(X_train_resize) // batch_size * epochs, max_rate=0.002,start_rate=0.0003)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_best_lrsch_onecycle_aug_dropout_128_he_irb6_512_epoch60.h5",save_best_only=True)

history=smv2_l.fit(X_train,y_train,
                   validation_data=(X_valid_final,y_valid),batch_size = batch_size, 
                   epochs=epochs,callbacks=[tboard_callback, check_best_cb,onecycle])

Epoch 1/60
8393/8393 [==============================] - 139s 16ms/step - loss: 1.6683 - accuracy: 0.5438 - val_loss: 0.5014 - val_accuracy: 0.8229
Epoch 2/60
8393/8393 [==============================] - 132s 16ms/step - loss: 0.5191 - accuracy: 0.8166 - val_loss: 0.4127 - val_accuracy: 0.8481
Epoch 3/60
8393/8393 [==============================] - 134s 16ms/step - loss: 0.4496 - accuracy: 0.8368 - val_loss: 0.4010 - val_accuracy: 0.8510
Epoch 4/60
8393/8393 [==============================] - 134s 16ms/step - loss: 0.4219 - accuracy: 0.8448 - val_loss: 0.4136 - val_accuracy: 0.8440
Epoch 5/60
8393/8393 [==============================] - 135s 16ms/step - loss: 0.4064 - accuracy: 0.8499 - val_loss: 0.3899 - val_accuracy: 0.8553
Epoch 6/60
8393/8393 [==============================] - 133s 16ms/step - loss: 0.3964 - accuracy: 0.8528 - val_loss: 0.3918 - val_accuracy: 0.8557
Epoch 7/60
8393/8393 [==============================] - 133s 16ms/step - loss: 0.3876 - accuracy: 0.8559 - val_loss: 0

8393/8393 [==============================] - 134s 16ms/step - loss: 0.3563 - accuracy: 0.8650 - val_loss: 0.3806 - val_accuracy: 0.8618
Epoch 28/60
8393/8393 [==============================] - 134s 16ms/step - loss: 0.3554 - accuracy: 0.8659 - val_loss: 1.0129 - val_accuracy: 0.7462
Epoch 29/60
8393/8393 [==============================] - 133s 16ms/step - loss: 0.3515 - accuracy: 0.8669 - val_loss: 0.3816 - val_accuracy: 0.8590
Epoch 30/60
8393/8393 [==============================] - 133s 16ms/step - loss: 0.3489 - accuracy: 0.8679 - val_loss: 0.7852 - val_accuracy: 0.7834
Epoch 31/60
8393/8393 [==============================] - 133s 16ms/step - loss: 0.3459 - accuracy: 0.8687 - val_loss: 0.5842 - val_accuracy: 0.8189
Epoch 32/60
8393/8393 [==============================] - 134s 16ms/step - loss: 0.3419 - accuracy: 0.8695 - val_loss: 0.3742 - val_accuracy: 0.8631
Epoch 33/60
8393/8393 [==============================] - 134s 16ms/step - loss: 0.3387 - accuracy: 0.8708 - val_loss: 0.3715

8393/8393 [==============================] - 131s 16ms/step - loss: 0.2742 - accuracy: 0.8905 - val_loss: 0.3465 - val_accuracy: 0.8735
Epoch 54/60
8393/8393 [==============================] - 132s 16ms/step - loss: 0.2712 - accuracy: 0.8916 - val_loss: 0.3460 - val_accuracy: 0.8754
Epoch 55/60
8393/8393 [==============================] - 134s 16ms/step - loss: 0.2678 - accuracy: 0.8925 - val_loss: 0.3511 - val_accuracy: 0.8748
Epoch 56/60
8393/8393 [==============================] - 132s 16ms/step - loss: 0.2656 - accuracy: 0.8935 - val_loss: 0.3433 - val_accuracy: 0.8776
Epoch 57/60
8393/8393 [==============================] - 134s 16ms/step - loss: 0.2625 - accuracy: 0.8946 - val_loss: 0.3431 - val_accuracy: 0.8778
Epoch 58/60
8393/8393 [==============================] - 134s 16ms/step - loss: 0.2603 - accuracy: 0.8954 - val_loss: 0.3453 - val_accuracy: 0.8772
Epoch 59/60
8393/8393 [==============================] - 134s 16ms/step - loss: 0.2582 - accuracy: 0.8959 - val_loss: 0.3451

smv2l_adam_best_lrsch_onecycle_aug_dropout_128_he_irb6_256_epoch60.h5를 smv2l_adam_best_lrsch_onecycle_aug_dropout_t4_128_he_irb6_512_epoch60.h5로 이름 바꿔서 사용

In [11]:
    #황성현 전민재 IRB61024, maxlr 0.003
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=2,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=4,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=4,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=4,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
    batch_size=128
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [ ]:
#황성현 전민재
from datetime import datetime
K = keras.backend
epochs=40
logs = "logs/" + datetime.now().strftime("SMV2L_lrsch_onecycle_dropout50_128_he_irb6_1024_epoch40%Y%m%d-%H%M%S")
onecycle = OneCycleScheduler(len(X_train_resize) // batch_size * epochs, max_rate=0.003,start_rate=0.0003)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_best_lrsch_onecycle_aug_dropout50_128_he_irb6_1024_t4_epoch40.h5",save_best_only=True)

history=smv2_l.fit(X_train,y_train,
                   validation_data=(X_valid_final,y_valid),batch_size = batch_size, 
                   epochs=epochs,callbacks=[tboard_callback, check_best_cb,onecycle])